## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [1]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
from pysus import SIH
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geobr
import os

from merradownload.merra_scraping import baixar_merra

## 01 - Baixar shapefiles

In [2]:
municipios_coord_df = pd.read_csv(r"../data/utils/municipios_coord.csv", sep=";")
estados = sorted(municipios_coord_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(municipios_coord_df[municipios_coord_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(options=estados, description='Estado:')
municipios_dd = Dropdown(options=municipios_por_estado[estados[0]], description='Município:')
baixar_btn = Button(description="Baixar e Plotar Shapefile")
output_mapa = Output()

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

def baixar_plotar_shapefile(b):
    uf = estados_dd.value
    municipio = municipios_dd.value

    localizar_municipio = geobr.lookup_muni(name_muni=municipio)
    localizar_municipio = localizar_municipio[localizar_municipio['abbrev_state'] == uf]

    output_mapa.clear_output()
    
    with output_mapa:
        print(f"Baixando shapefile para {municipio}, {uf}...")
        code_muni = localizar_municipio.iloc[0]['code_muni']
        gdf = geobr.read_municipality(code_muni=code_muni)
        print("Download concluído.")

        save_dir = rf"../data/shapefiles/{uf}-{municipio.replace(' ', '_')}/"
        os.makedirs(save_dir, exist_ok=True)
        
        # Salva o shapefile
        filepath = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}.shp")
        gdf.to_file(filepath, driver='ESRI Shapefile')
        print(f"Shapefile salvo em: {filepath}")

        # Salva o CSV com código IBGE
        try:
            ibge_df = pd.read_csv(r"../data/utils/municipios_ibge.csv")
            # Tenta encontrar pelo código IBGE (garantindo que ambos sejam do mesmo tipo, ex: float/int)
            # O geobr retorna float, o csv pode ter int.
            city_ibge_info = ibge_df[ibge_df['codigo_ibge'] == code_muni]
            
            if city_ibge_info.empty:
                # Fallback: Tenta pelo nome do município
                city_ibge_info = ibge_df[ibge_df['Nome_Município'] == municipio]
            
            if not city_ibge_info.empty:
                csv_path = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}_ibge.csv")
                city_ibge_info.to_csv(csv_path, index=False)
                print(f"CSV IBGE salvo em: {csv_path}")
            else:
                print("Aviso: Não foi possível encontrar informações no arquivo municipios_ibge.csv")
        except Exception as e:
            print(f"Erro ao salvar CSV IBGE: {e}")

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        gdf.plot(ax=ax, facecolor='#add8e6', edgecolor='black')
        ax.set_title(f"Mapa de {municipio} - {uf}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.grid(True)
        plt.show()

baixar_btn.on_click(baixar_plotar_shapefile)

display(VBox([estados_dd, municipios_dd, baixar_btn, output_mapa]))

## 02 - Criar arquivo final

In [3]:
# UI para gerar o arquivo final chamando download_all(...)
import sys
from datetime import date
from pathlib import Path
import pandas as pd
from ipywidgets import (
    VBox, HBox, Button, Output, Text, Dropdown, Checkbox, DatePicker,
    Layout,
 )
from IPython.display import display

# Garantir import do módulo src/baixar_dados/download_all.py (notebook roda em src/)
if "baixar_dados" not in sys.path:
    sys.path.insert(0, "baixar_dados")
from download_all import download_all

out = Output()

mode_dd = Dropdown(
    options=[("Pasta (todos os shapefiles)", "dir"), ("Um shapefile", "single")],
    value="dir",
    description="Modo:",
    layout=Layout(width="320px"),
 )

shapefiles_dir_txt = Text(
    value="../data/shapefiles",
    description="Shapefiles dir:",
    layout=Layout(width="700px"),
 )

shapefile_txt = Text(
    value="",
    description="Shapefile (.shp):",
    placeholder="Ex: ../data/shapefiles/SP-Diadema/SP_Diadema.shp",
    layout=Layout(width="700px"),
 )

start_dp = DatePicker(description="Início:", value=date(2024, 1, 1), layout=Layout(width="220px"))
end_dp = DatePicker(description="Fim:", value=date(2024, 12, 31), layout=Layout(width="220px"))

output_dir_txt = Text(value="../data/output", description="Output dir:", layout=Layout(width="520px"))
cache_dir_txt = Text(value="../data/cache", description="Cache dir:", layout=Layout(width="520px"))
final_csv_txt = Text(
    value="../data/output/final/final_by_ibge_date.csv",
    description="Final CSV:",
    layout=Layout(width="700px"),
 )

write_per_muni_cb = Checkbox(value=True, description="Salvar final por município")
write_final_cb = Checkbox(value=True, description="Salvar final unificado")
log_level_dd = Dropdown(options=["DEBUG", "INFO", "WARNING", "ERROR"], value="INFO", description="Log:")

run_btn = Button(description="Baixar e criar arquivo final", button_style="success")

def _toggle_mode(*_):
    if mode_dd.value == "dir":
        shapefiles_dir_txt.layout.display = ""
        shapefile_txt.layout.display = "none"
    else:
        shapefiles_dir_txt.layout.display = "none"
        shapefile_txt.layout.display = ""

mode_dd.observe(_toggle_mode, names="value")
_toggle_mode()

def _run(_):
    out.clear_output()
    with out:
        if start_dp.value is None or end_dp.value is None:
            print("Selecione start e end.")
            return
        kwargs = dict(
            start=start_dp.value,
            end=end_dp.value,
            output_dir=output_dir_txt.value,
            cache_dir=cache_dir_txt.value,
            final_csv=final_csv_txt.value,
            write_per_municipio=write_per_muni_cb.value,
            write_final=write_final_cb.value,
            log_level=log_level_dd.value,
        )
        if mode_dd.value == "dir":
            kwargs["shapefiles_dir"] = shapefiles_dir_txt.value
        else:
            if not shapefile_txt.value.strip():
                print("Informe o caminho do shapefile (.shp).")
                return
            kwargs["shapefile"] = shapefile_txt.value.strip()

        print("Rodando download_all(...) com:")
        for k, v in kwargs.items():
            print(f"- {k}={v}")

        df = download_all(**kwargs)
        if df is None or df.empty:
            print("\nNenhum dado retornado (todos os downloaders podem ter falhado ou não havia dados no período).")
            return

        print("\nOK. Preview do DataFrame final:")
        display(df.head())
        print(f"\nLinhas: {len(df)} | Colunas: {len(df.columns)}")
        if write_final_cb.value:
            print(f"CSV final salvo em: {Path(final_csv_txt.value).resolve()}")

run_btn.on_click(_run)

ui = VBox([
    HBox([mode_dd, log_level_dd]),
    shapefiles_dir_txt,
    shapefile_txt,
    HBox([start_dp, end_dp]),
    HBox([output_dir_txt, cache_dir_txt]),
    final_csv_txt,
    HBox([write_per_muni_cb, write_final_cb]),
    run_btn,
    out,
])

display(ui)